In [1]:
import os
import torch
import numpy as np
import cv2

from tqdm import tqdm
from pathlib import Path
from EvEye.utils.tonic.functional.ToFrameStack import to_frame_stack_numpy
from EvEye.utils.tonic.slicers.SliceEventsAtIndices import slice_events_at_timepoints
from EvEye.utils.processor.TxtProcessor import TxtProcessor
from EvEye.utils.visualization.visualization import (
    visualize,
    save_image,
    load_image,
    draw_contour,
)
from tqdm import tqdm
from natsort import natsorted
from EvEye.model.model_factory import make_model
from EvEye.utils.scripts.process_model_output import process_model_output
from EvEye.utils.scripts.load_config import load_config
import albumentations as A
from albumentations.pytorch import ToTensorV2

/home/junyuan/anaconda3/envs/EvEye/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
time_window = 10000
sensor_size = (346, 260, 2)

In [3]:
output_base_path = Path("/mnt/data2T/junyuan/eye-tracking/outputs/EventMasks")
root_txt_path = Path(
    "/mnt/data2T/junyuan/eye-tracking/datasets/DavisEyeCenterDataset/test/data"
)
root_rgb_path = Path(
    "/mnt/data2T/junyuan/eye-tracking/datasets/DavisEyeCenterDatasetFrames"
)
user_paths = natsorted([f for f in root_rgb_path.iterdir() if f.is_dir()])

In [4]:
config_path = Path("/mnt/data2T/junyuan/eye-tracking/configs/OutputGroundTruth.yaml")
config = load_config(config_path)
model = make_model(config["model"])
model.load_state_dict(
    torch.load(config["test"]["ckpt_path"], map_location="cuda:0")["state_dict"]
)
model = model.cuda()
model.eval()

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

In [5]:
transform = A.Compose(
    [
        A.Resize(height=240, width=346),
        A.Normalize(
            mean=[0.0],
            std=[1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

In [6]:
for user_path in user_paths:
    print(f"Processing {user_path.stem}...")
    rgb_path = natsorted(list(user_path.glob("*.png")))
    txt_path = Path(f"{root_txt_path}/{user_path.stem}_events.txt")
    output_path = output_base_path / user_path.stem
    os.makedirs(output_path, exist_ok=True)
    if not txt_path.exists():
        continue
    events = TxtProcessor(txt_path).load_events_from_txt()
    events_end_time = events['t'][-1]
    events_start_time = events['t'][0]
    num_frames = (events_end_time - events_start_time) // time_window
    count = 0
    for rgb in tqdm(rgb_path):
        end_time = int(rgb.stem.split("_")[1])
        start_time = end_time - time_window
        event_segment = slice_events_at_timepoints(events, start_time, end_time)
        if len(event_segment) < 1000:
            continue
        event_frame = to_frame_stack_numpy(
            event_segment,
            sensor_size,
            1,
            "causal_linear",
            start_time,
            end_time,
        )
        event_frame_vis = visualize(event_frame)
        event_frame_vis_name = str(output_path / f"{count:05}_{end_time}.png")
        save_image(event_frame_vis, event_frame_vis_name)
        image_gray = load_image(str(rgb), "grayscale")[0]
        transformed = transform(image=image_gray)
        image = transformed["image"]
        image = image.unsqueeze(0)
        with torch.no_grad():
            output = model(image.cuda())
        mask = process_model_output(output, use_softmax=True)
        mask = mask.detach().cpu().numpy()  # shape: [260, 346]
        mask = cv2.resize(
            mask,
            (sensor_size[0], sensor_size[1]),
            interpolation=cv2.INTER_NEAREST,
        )
        mask_vis = draw_contour(mask)
        save_image(mask_vis, str(output_path / f"{count:05}_{end_time}_mask.png"))
        image_name = str(output_path / f"{count:05}_{end_time}_rgb.png")
        save_image(image_gray, image_name)
        count += 1

Processing user1_left_session_1_0_1...
Processing user1_left_session_1_0_2...
Processing user1_left_session_2_0_1...
Processing user1_left_session_2_0_2...
Processing user1_right_session_1_0_1...
Processing user1_right_session_1_0_2...
Processing user1_right_session_2_0_1...
Processing user1_right_session_2_0_2...
Processing user2_left_session_1_0_1...
Processing user2_left_session_1_0_2...
Processing user2_left_session_2_0_1...
Processing user2_left_session_2_0_2...
Processing user2_right_session_1_0_1...
Processing user2_right_session_1_0_2...
Processing user2_right_session_2_0_1...
Processing user2_right_session_2_0_2...
Processing user3_left_session_1_0_1...
Processing user3_left_session_1_0_2...
Processing user3_left_session_2_0_1...
Processing user3_left_session_2_0_2...
Processing user3_right_session_1_0_1...
Processing user3_right_session_1_0_2...
Processing user3_right_session_2_0_1...
Processing user3_right_session_2_0_2...
Processing user4_left_session_1_0_1...
Processing us

100%|██████████| 5010/5010 [00:08<00:00, 622.52it/s]


Processing user43_left_session_1_0_2...


100%|██████████| 2843/2843 [00:04<00:00, 581.85it/s]


Processing user43_left_session_2_0_1...


...

100%|██████████| 2794/2794 [00:04<00:00, 666.96it/s]


Processing user48_left_session_1_0_1...


100%|██████████| 5060/5060 [00:10<00:00, 502.87it/s]


Processing user48_left_session_1_0_2...


100%|██████████| 2821/2821 [00:06<00:00, 441.81it/s]
